**PASO 1. PREPARACION DE DATOS**

In [2]:
# Importar librerias
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [3]:
# Cargar los datos en un DataFrame
data = pd.read_excel("riesgos_dataset.xlsx")

In [4]:
# ¿CUANTAS FILAS Y COLUMNAS HAY EN EL DATASET COMPLETO?
num_filas, num_columnas = data.shape
# Imprimir el resultado
print(f"El DataFrame tiene {num_filas} filas y {num_columnas} columnas.")

El DataFrame tiene 252857 filas y 14 columnas.


In [5]:
# DEL DF ORIGINAL EXTRAEMOS TODAS LAS ETIQUETAS DE RIESGOS
# De esta forma sabremos a que se corresponde cada riesgo
data_etiq_ixRiesgo = data[['ixRiesgo', 'sAnalisis']].copy()
data_etiq_ixRiesgo.drop_duplicates(subset=['ixRiesgo'], inplace=True)
data_etiq_ixRiesgo = data_etiq_ixRiesgo.sort_values(by='ixRiesgo')
data_etiq_ixRiesgo.reset_index(drop=True, inplace=True)
data_etiq_ixRiesgo.head(6)

,ixRiesgo,sAnalisis
0,1,Accidentes de circulación durante la jornada l...
1,2,Trabajos que implican el trato con personas aj...
2,3,Atrapamiento por o entre objetos
3,4,Atrapamiento por elementos móviles de equipos
4,5,Atrapamiento por vuelco de máquinas o vehículos
5,6,Atropellos o golpes con vehículos


In [6]:
# AL IGUAL QUE EL ANTERIOR EXTRAEMOS TODAS LAS ETIQUETAS DEL SECTOR
data_etiq_sector = data[['nSectorEmp', 'sectorEmpresa']].copy()
data_etiq_sector.drop_duplicates(subset=['nSectorEmp'], inplace=True)
data_etiq_sector['sectorEmpresa'] = data_etiq_sector['sectorEmpresa'].str.replace(r'^\d+\.\s', '', regex=True)
data_etiq_sector = data_etiq_sector.sort_values(by='nSectorEmp')
data_etiq_sector.reset_index(drop=True, inplace=True)
data_etiq_sector

,nSectorEmp,sectorEmpresa
0,1,Oficinas y despachos
1,2,Construcción
2,4,Industria y diversos
3,5,Centros Sanitarios
4,6,Transporte
5,7,Centros Educativos
6,8,Mantenimiento Industrial
7,9,Hostelería
8,10,Otros
9,11,Servicios


In [7]:
# FUNCION PARA SABER QUE SIGNIFICADO TIENE CADA RIESGO
def obtener_etiq_ixRiesgo(valor_ixRiesgo):
    # Buscar el sAnalisis correspondiente a ixRiesgo
    etiqueta = data_etiq_ixRiesgo[data_etiq_ixRiesgo['ixRiesgo'] == valor_ixRiesgo]['sAnalisis'].values
    if len(etiqueta) > 0:
        return etiqueta[0]
    else:
        return "Etiqueta no encontrada"  # En caso de que ixRiesgo no exista

# Ejemplo de uso:
etiqueta = obtener_etiq_ixRiesgo(104)
print(etiqueta)

Quemaduras por contacto con superficies calientes, superficies frías, llama, etc.


In [8]:
# FUNCION PARA OBTENER LA ETIQUETA DEL SECTOR
def obtener_etiq_sector(sector):
    # Buscar el sectorEmpresa correspondiente al número de sector
    etiqueta = data_etiq_sector[data_etiq_sector['nSectorEmp'] == sector]['sectorEmpresa'].values
    if len(etiqueta) > 0:
        # Eliminar el número y el punto y devolver solo el texto
        return etiqueta[0]
    else:
        return "Sector no encontrado"  # En caso de que el sector no exista

# Ejemplo de uso:
sector_texto = obtener_etiq_sector(2)  
print(sector_texto)

Construcción


In [9]:
# UNA VEZ TENEMOS LAS ETIQUETAS DE RIESGO Y DEL SECTOR 
# COMENZAMOS A DEPURAR EL DATAFRAME ORIGINAL

# ELIMINAR COLUMNAS SOBRANTES
# Seleccionamos las columnas que deseamos, el resto las eliminamos
columnas_deseadas = ['nSectorEmp', 'sCodCliente', 'idEvaluacion', 'ixRiesgo', 'ixPuesto', 'ixFuenteRL']
# Seleccionar solo las columnas deseadas
data = data[columnas_deseadas]
# El DataFrame ahora contiene solo las columnas deseadas

In [10]:
# VOLVEMOS A COMPROBAR EL TAMAÑO DEL DATAFRAME
# Comprobaremos que hemos pasado de 14 columnas a 6 columnas
num_filas, num_columnas = data.shape
# Imprimir el resultado
print(f"El DataFrame tiene {num_filas} filas y {num_columnas} columnas.")

El DataFrame tiene 252857 filas y 6 columnas.


In [11]:
# CONTEO DE DUPLICADOS
# Comprobaremos que mas de la mitad son duplicados 
num_duplicados = data.duplicated().sum()
# Imprimir el resultado
print(f"El DataFrame tiene {num_duplicados} filas duplicadas.")

El DataFrame tiene 127289 filas duplicadas.


In [12]:
# ELIMINAMOS LOS DUPLICADOS
data = data.drop_duplicates(subset=columnas_deseadas, keep='first')

In [13]:
# VOLVEMOS A COMPROBAR EL TAMAÑO DEL DATAFRAME
num_filas, num_columnas = data.shape
# Imprimir el resultado
print(f"El DataFrame tiene {num_filas} filas y {num_columnas} columnas.")

El DataFrame tiene 125568 filas y 6 columnas.


In [14]:
# VERIFICACION DE VALORES FALTANTES
missing_values = data.isnull().sum()
print(missing_values)

nSectorEmp      0
sCodCliente     0
idEvaluacion    0
ixRiesgo        0
ixPuesto        0
ixFuenteRL      0
dtype: int64


In [15]:
# RESETEAMOS EL INDEX
data = data.reset_index(drop=True)
data.head()

,nSectorEmp,sCodCliente,idEvaluacion,ixRiesgo,ixPuesto,ixFuenteRL
0,2,3,2,1,1,0
1,2,3,2,2,1,0
2,2,3,2,2,2,0
3,2,3,2,3,2,0
4,2,3,2,5,2,0


In [16]:
# ALMACENAR EL DATAFRAME RESULTANTE EN UN EXCEL
# nombre_archivo = 'riesgos_dataset_sin_duplicados.xlsx'
# sheet_name = 'mi_hoja'

# data.to_excel(nombre_archivo, sheet_name=sheet_name, index=False)
print("Consulte el archivo riesgos_dataset_sin_duplicados.xlsx para ver el dataframe resultante")

Consulte el archivo riesgos_dataset_sin_duplicados.xlsx para ver el dataframe resultante


**PASO 2. ANALISIS DE DATOS Y EXTRACCION DE ESTADISTICAS**

In [17]:
# 1. CUANTOS SECTORES APARECEN EN EL DATASET
num_sectores = data['nSectorEmp'].nunique()
print(f"El número de sectores distintos en el DataFrame es: {num_sectores}")

El número de sectores distintos en el DataFrame es: 11


In [18]:
# 2. LISTAR TODOS LOS SECTORES
sectores_unicos = data['nSectorEmp'].unique()
sectores_unicos_ordenados = sorted(sectores_unicos)
for elemento in sectores_unicos_ordenados:
    sector = obtener_etiq_sector(elemento)
    print("Sector {}: {}".format(elemento, sector))

Sector 1: Oficinas y despachos
Sector 2: Construcción
Sector 4: Industria y diversos
Sector 5: Centros Sanitarios
Sector 6: Transporte
Sector 7: Centros Educativos
Sector 8: Mantenimiento Industrial
Sector 9: Hostelería
Sector 10: Otros
Sector 11: Servicios
Sector 12: Sector Metal


In [19]:
# 3. CUANTOS CLIENTES HAY EN LA BASE DE DATOS
num_clientes = data['sCodCliente'].nunique()
print(f"El número de clientes distintos en la base de datos es: {num_clientes}")

El número de clientes distintos en la base de datos es: 934


In [20]:
# 4. CUANTOS TIPOS DE RIESGO (ixRiesgo) HAY EN EL DATASET
tipos_distintos_de_riesgo = data['ixRiesgo'].nunique()
print(f"El número de tipos distintos de ixRiesgo es: {tipos_distintos_de_riesgo}")

El número de tipos distintos de ixRiesgo es: 107


In [21]:
# 5. POR CADA CLIENTE, CUANTAS VECES APARECE CADA RIESGO
# Aqui podemos consultar los riesgos que mas se repiten en cada cliente
# en orden descendente
print("Abrir archivo 'resultados_riesgos_por_cliente.xlsx' y filtrar por cliente")
print()
# Creamos un df agrupando por cliente y riesgo, contamos los registros y reseteamos el index
conteo_riesgo_por_cliente = data.groupby(['sCodCliente', 'ixRiesgo']).size().reset_index(name='conteo')

# Ordenamos el resultado de mayor a menor. El riesgo que mas se repite, en primer lugar
conteo_riesgo_por_cliente = conteo_riesgo_por_cliente.sort_values(by=['sCodCliente', 'conteo'], ascending=[True, False])

# Crear una lista para almacenar los datos
resultados = []

# Recorremos todo el df y vamos obteniendo las etiquetas del riesgo
for index, row in conteo_riesgo_por_cliente.iterrows():
    cliente = row['sCodCliente']
    riesgo = row['ixRiesgo']
    conteo = row['conteo']
    etiqueta = obtener_etiq_ixRiesgo(riesgo)
    
    # Agregar la información a la lista
    resultados.append({'CLIENTE': cliente, 'RIESGO': etiqueta, 'VECES': conteo})

    # Mostramos la informacion en pantalla
    # print(f'Al cliente {cliente} el riesgo {riesgo}\n{etiqueta}\nle aparece {conteo} veces')
    # print("")

# Y almacenamos los resultados en un df para poderlos pasar a una excel
# resultados_df = pd.DataFrame(resultados)

# Guardar el DataFrame de resultados en un archivo Excel
# resultados_df.to_excel("resultados_riesgos_por_cliente.xlsx", index=False)

Abrir archivo 'resultados_riesgos_por_cliente.xlsx' y filtrar por cliente



In [22]:
# 6. POR CADA CLIENTE ¿CUAL ES EL RIESGO QUE MAS SE REPITE?
# Obtener el riesgo más común por cliente
riesgo_mas_comun_por_cliente = data.groupby('sCodCliente')['ixRiesgo'].agg(lambda x: x.value_counts().idxmax())

# Mapear los números a las etiquetas utilizando la función obtener_etiq_ixRiesgo
riesgo_mas_comun_por_cliente = riesgo_mas_comun_por_cliente.map(obtener_etiq_ixRiesgo)

# Imprimir el resultado
print(riesgo_mas_comun_por_cliente)


sCodCliente
0004        Caídas al mismo nivel por suelos resbaladizos ...
0019                         Choches contra objetos inmóviles
0020        Posible exposición, sea consciente o no, a con...
0021        Tareas que implican ciclos de trabajo con movi...
0022                            Contactos eléctricos directos
                                  ...                        
CAAEE005    Tareas que pueden ocasionar caídas a distinto ...
CAAEE006    Tareas que pueden ocasionar caídas a distinto ...
CAAEE007    Tareas que pueden ocasionar caídas a distinto ...
CAAEE008    Tareas que pueden ocasionar caídas a distinto ...
CAAEE009                        Contactos eléctricos directos
Name: ixRiesgo, Length: 934, dtype: object


In [23]:
# 7. NUMERO DE RIESGOS EN CADA SECTOR
# Recorremos todos los sectores
for numero_sector in sectores_unicos_ordenados:

    # Filtrar el DataFrame para obtener solo las filas correspondientes al sector deseado.
    datos_sector = data[data['nSectorEmp'] == numero_sector]

    # Agrupar por nSectorEmp e ixRiesgo y contar las ocurrencias
    conteo_riesgos = datos_sector.groupby(['nSectorEmp', 'ixRiesgo']).size().reset_index(name='conteo')

    # Lo almacenamos en un excel para analizarlo, ya que aqui la salida sale truncada
    # conteo_riesgos.to_excel("riesgos_sector_{}.xlsx".format(numero_sector), index=False)

    sector = obtener_etiq_sector(numero_sector)

    # Imprimir el resultado    
    print("Nº riesgos sector", numero_sector, sector,":", len(conteo_riesgos))        

Nº riesgos sector 1 Oficinas y despachos : 57
Nº riesgos sector 2 Construcción : 66
Nº riesgos sector 4 Industria y diversos : 77
Nº riesgos sector 5 Centros Sanitarios : 50
Nº riesgos sector 6 Transporte : 56
Nº riesgos sector 7 Centros Educativos : 54
Nº riesgos sector 8 Mantenimiento Industrial : 50
Nº riesgos sector 9 Hostelería : 69
Nº riesgos sector 10 Otros : 29
Nº riesgos sector 11 Servicios : 81
Nº riesgos sector 12 Sector Metal : 49


In [24]:
# 8.1. RIESGOS MAS FRECUENTES EN CADA SECTOR (FORMATO EXCEL)
# Agrupar por sector y riesgo y contar la frecuencia
frecuencia_riesgos_por_sector = data.groupby(['nSectorEmp', 'ixRiesgo']).size().reset_index(name='frecuencia')

# Ordenar los datos por sector y frecuencia
frecuencia_riesgos_ordenados = frecuencia_riesgos_por_sector.sort_values(by=['nSectorEmp', 'frecuencia'], ascending=[True, False])

# Mostrar el DataFrame ordenado
print(frecuencia_riesgos_ordenados)
# Exportamos a excel
# frecuencia_riesgos_ordenados.to_excel("Frecuencia_riesgos_x_sector.xlsx", index=False)

     nSectorEmp  ixRiesgo  frecuencia
32            1        78         354
34            1        92         344
42            1       154         334
27            1        66         331
11            1        24         329
..          ...       ...         ...
616          12        71           3
632          12       255           3
633          12       257           3
637          12       261           3
636          12       260           2

[638 rows x 3 columns]


In [25]:
# 8.2. RIESGOS MAS FRECUENTES EN CADA SECTOR (FORMATO GRAFICAS)

# frecuencia_riesgos_ordenados = frecuencia_riesgos_por_sector

# # Obténemos la lista de sectores únicos
# sectores_unicos = frecuencia_riesgos_por_sector['nSectorEmp'].unique()

# # Iteramos a través de cada sector y creamos una gráfica
# for sector in sectores_unicos:
#     datos_sector = frecuencia_riesgos_por_sector[frecuencia_riesgos_por_sector['nSectorEmp'] == sector]
    
#     plt.figure(figsize=(10, 6))  # Ajusta el tamaño de la gráfica
#     plt.bar(datos_sector['ixRiesgo'], datos_sector['frecuencia'])
#     plt.xlabel('Riesgo')
#     plt.ylabel('Frecuencia')
#     plt.title(f'Frecuencia de Riesgos en el Sector {sector}')
    
#     # Agregar el número de riesgo encima de cada barra
#     for x, y in zip(datos_sector['ixRiesgo'], datos_sector['frecuencia']):
#         plt.text(x, y+5, str(x), fontsize=8, ha='center', va='bottom', rotation='vertical')
#     plt.savefig(f'Frecuencia_riesgos_sector_{sector}.png')
#     plt.show()  # Muestra la gráfica

In [26]:
# 9. RIESGOS QUE APARECEN EN UNA EMPRESA DETERMINADA, 
# PERO NO ACOSTUMBRAN A OCURRIR (RIESGOS POCO FRECUENTES)
# Filtrar datos para la empresa seleccionada
empresa_evaluada = '0019'
datos_empresa_seleccionada = data[data['sCodCliente'] == empresa_evaluada]

# Definir un umbral de frecuencia para considerar un riesgo como poco común
umbral_poco_comun = 3  

# Obtener la lista de riesgos poco comunes en las evaluaciones de la empresa 1072
riesgos_poco_comunes = datos_empresa_seleccionada['ixRiesgo'].value_counts()[datos_empresa_seleccionada['ixRiesgo'].value_counts() < umbral_poco_comun].index

# Mostrar la lista de riesgos poco comunes
print("RIESGOS POCO COMUNES EN LAS EVALUACIONES DE LA EMPRESA {}:".format(empresa_evaluada))
for elemento in riesgos_poco_comunes:
    riesgo_poco_comun = obtener_etiq_ixRiesgo(elemento)
    print(f"- Riesgo {elemento}: {riesgo_poco_comun}")

RIESGOS POCO COMUNES EN LAS EVALUACIONES DE LA EMPRESA 0019:
- Riesgo 218: Posible exposición, sea consciente o no, a contaminantes biológicos.
- Riesgo 104: Quemaduras por contacto con superficies calientes, superficies frías, llama, etc.
- Riesgo 106: Contactos con sustancias cáusticas y/o corrosivas


In [27]:
# 10. RIESGOS MAS COMUNES EN UNA EMPRESA DETERMINADA
# Aqui se muestran todos los riesgos que aparecen en una empresa determinada, en orden descendente,
# es decir, el riesgo que mas veces se repite aparece en primer lugar y el que menos se repite, en ultimo lugar.
empresa_evaluada = '0019'
datos_empresa = data[data['sCodCliente'] == empresa_evaluada]

# Obtener la lista de riesgos más comunes en las evaluaciones de la empresa 1072 y ordenarla en orden ascendente
riesgos_mas_comunes = datos_empresa['ixRiesgo'].value_counts().index

# Mostrar la lista de riesgos más comunes en orden ascendente
print("Riesgos más comunes en las evaluaciones de la empresa {} (orden descendente):".format(empresa_evaluada))

# Definir la cantidad de elementos a mostrar
num_riesgos_a_mostrar = 5  # Cambia este valor al número deseado

for elemento in riesgos_mas_comunes:
    if num_riesgos_a_mostrar > 0:
        riesgo_mas_comun = obtener_etiq_ixRiesgo(elemento)
        print(f"- Riesgo {elemento}: {riesgo_mas_comun}")
        num_riesgos_a_mostrar -= 1
    else:
        break  # Sal del bucle cuando se muestren la cantidad deseada de elementos



Riesgos más comunes en las evaluaciones de la empresa 0019 (orden descendente):
- Riesgo 92: Choches contra objetos inmóviles
- Riesgo 8: Caída de objetos por desplome o derrumbamiento
- Riesgo 9: Tareas que pueden ocasionar caídas a distinto nivel, para las que no es necesario disponer de formación en trabajos en altura.
- Riesgo 78: Caídas al mismo nivel por suelos resbaladizos o obstáculos en zonas de paso
- Riesgo 154: Mantenimiento de posturas forzadas o estáticas durante el trabajo.


**PASO 3. OBTENCION DE LA SIMILITUD ENTRE UNA EVALUACION DE UNA EMPRESA DETERMINADA Y TODAS LAS EVALUACIONES DE TODAS LAS EMPRESAS DEL MISMO SECTOR**

**PASO 3.1. Obtencion de datos**

In [34]:
# SELECCIONE LA EMPRESA QUE DESEA EVALUAR
empresa_evaluada = '0019'

# EXTRAEMOS INFORMACION DE LA EMPRESA EVALUADA (EVALUACIONES Y SECTOR)
# Filtramos todas las evaluaciones de la empresa
evaluaciones_empresa = data[data['sCodCliente'] == empresa_evaluada]
# Contamos el numero de evaluaciones que ha tenido
num_evaluaciones = evaluaciones_empresa['idEvaluacion'].nunique()
# Obtenemos la lista de dichas evaluaciones
evaluaciones_id = evaluaciones_empresa['idEvaluacion'].unique()

# COMPROBAR A QUE SECTOR PERTENECE DICHA EMPRESA
# Filtrar el DataFrame para encontrar la fila correspondiente a la empresa de interés
empresa_fila = data[data['sCodCliente'] == empresa_evaluada]
# Obtener el sector de la empresa de interés
sector_de_la_empresa = empresa_fila['nSectorEmp'].iloc[0]

# Imprimir la informacion
print("La empresa {} ha pasado {} evaluaciones.".format(empresa_evaluada, num_evaluaciones))
print("Las idEvaluacion de las evaluaciones son:", evaluaciones_id)
print(f"La empresa {empresa_evaluada} pertenece al sector {sector_de_la_empresa}")

La empresa 0019 ha pasado 6 evaluaciones.
Las idEvaluacion de las evaluaciones son: [1 2 3 4 5 6]
La empresa 0019 pertenece al sector 11


In [35]:
# OBTENCION DE LA LISTA DE RIESGOS DE LA EMPRESA SELECCIONADA 
# EN 1 EVALUACION DETERMINADA
codigo_evaluacion = 6  # idEvaluacion      

# Filtramos los datos de la empresa seleccionada 
datos_empresa_evaluada = data[(data['sCodCliente'] == empresa_evaluada) &
                              (data['idEvaluacion'] == codigo_evaluacion)]

# Obtenemos la lista de riesgos (ixRiesgo) en dicha evaluacion y eliminamos duplicados
lista_riesgos_empresa_evaluada = list(set(datos_empresa_evaluada['ixRiesgo']))

# Imprimir las listas de riesgos
print("Lista de riesgos en la evaluación {} de la empresa {}:".format(codigo_evaluacion, empresa_evaluada))
print(lista_riesgos_empresa_evaluada)
print("Longitud lista: ", len(lista_riesgos_empresa_evaluada))

Lista de riesgos en la evaluación 6 de la empresa 0019:
[1, 2, 3, 257, 5, 6, 7, 8, 9, 135, 258, 259, 260, 16, 144, 23, 24, 153, 154, 31, 48, 53, 54, 55, 56, 58, 62, 66, 78, 92, 93, 104, 233, 106, 234]
Longitud lista:  35


In [36]:
# OBTENCION DE TODOS LOS RIESGOS DE TODAS LAS EVALUACIONES DE TODAS LAS EMPRESAS DEL MISMO SECTOR
# Filtrar el DataFrame para obtener los datos del sector deseado
datos_sector = data[data['nSectorEmp'] == sector_de_la_empresa]

# Obtenemos la lista de riesgos de todas las evaluaciones de todas las empresas del sector
lista_riesgos_todas_eval_sector = datos_sector['ixRiesgo'].unique()

# Imprimir la lista de riesgos
print("Lista de riesgos del sector", sector_de_la_empresa)
print(lista_riesgos_todas_eval_sector)
print("Longitud: ", len(lista_riesgos_todas_eval_sector))

Lista de riesgos del sector 11
[218   2   8   9  53  55  66  78  92 153 154  23   1  24  54   3 259 222
   6  31  67 233  42  49  47 135 234  26  64 140 142 240  52 104 106  12
  13  14 255  69 260   7  56  58  62  71  16  48 261   5  68  93 258 257
  57 144 133  30  37  39  43  50  59 223 242 230 241 254 256  25  20  29
  80 115   4  15  36  17  77 148 170]
Longitud:  81


**PASO 3.2. Normalizacion de datos**

In [37]:
# NORMALIZACION DE LAS LISTAS
# Listas de ejemplo (antes de la normalizacion):
# lista1_original = [1,2,3,4,11,5,6,7]
# lista2_original = [1,16,3,4,5,7]
# Resultado (despues de la normalizacion):
# lista1_normalizada: [1, 2, 3, 4, 5, 6, 7, 0, 0, 0, 11, 0, 0, 0, 0, 0]
# lista2_normalizada: [1, 0, 3, 4, 5, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 16]
# Numero de elementos de ambas listas igual al numero mayor de las dos listas en conjunto 
# y un 0 en aquellos lugares que no tienen elemento repetido en ambas listas.
# --------------------------------------------------------------------------------------------
# Buscamos el número mayor entre ambas listas
numero_mayor = max(max(lista_riesgos_empresa_evaluada), max(lista_riesgos_todas_eval_sector))

# Llenamos ambas listas con ceros en las posiciones correspondientes
lista1_normalizada = [0 if i not in lista_riesgos_empresa_evaluada else i for i in range(1, numero_mayor + 1)]
lista2_normalizada = [0 if i not in lista_riesgos_todas_eval_sector else i for i in range(1, numero_mayor + 1)]

# Observamos el resultado
print("lista1_normalizada (lista_riesgos_empresa_evaluada): ", lista1_normalizada)
print("lista2_normalizada (lista_riesgos_todas_eval_sector):", lista2_normalizada)
print("Longitud lista 1:", len(lista1_normalizada))
print("Longitud lista 2:", len(lista2_normalizada))

lista1_normalizada (lista_riesgos_empresa_evaluada):  [1, 2, 3, 0, 5, 6, 7, 8, 9, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 23, 24, 0, 0, 0, 0, 0, 0, 31, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 48, 0, 0, 0, 0, 53, 54, 55, 56, 0, 58, 0, 0, 0, 62, 0, 0, 0, 66, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 78, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 92, 93, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 104, 0, 106, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 135, 0, 0, 0, 0, 0, 0, 0, 0, 144, 0, 0, 0, 0, 0, 0, 0, 0, 153, 154, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 233, 234, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 257, 258, 259, 260, 0]
lista2_normalizada (lista_riesgos_todas_eval_sector): [1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 0, 12, 13, 14, 15, 16, 17, 0, 0, 20, 0

**PASO 3.3. Aplicacion del algoritmo cosine_similarity()**

Comparamos ambas listas, la lista de riesgos de la empresa evaluada (lista 1) 
y la lista de riesgos de todo un sector (lista 2), ambas ya normalizadas, y aplicando el algoritmo obtenemos el indice de similitud entre la empresa evaluada y las demas empresas del mismo sector

In [38]:
# CALCULO DE LA SIMILITUD DEL COSENO ENTRE LAS DOS LISTAS YA NORMALIZADAS
# ----------------------------------
# LISTAS DE EJEMPLO PARA PROBAR EL ALGORITMO. 
# Habilite cualquiera de los 3 ejemplos y vea el resultado

# Ejemplo 1. Listas iguales (100% de similitud)
# lista1_normalizada = [1,2,3,4,5]
# lista2_normalizada = [1,2,3,4,5]

# Ejemplo 2. Listas parecidas (~50%)
# lista1_normalizada = [1,2,3,4,5]
# lista2_normalizada = [1,2,3,0,0]

# Ejemplo 3. Listas distintas (0% de similitud)
# lista1_normalizada = [1,2,3,4,5]
# lista2_normalizada = [0,0,0,0,0]
# ----------------------------------

# Listas reales que vamos a comparar
lista_riesgos_empresa_evaluada = np.array(lista1_normalizada)
lista_riesgos_todas_eval_sector = np.array(lista2_normalizada)

# Adaptamos las listas a la dimension correcta 
lista_riesgos_empresa_evaluada = lista_riesgos_empresa_evaluada.reshape(1, -1)
lista_riesgos_todas_eval_sector = lista_riesgos_todas_eval_sector.reshape(1, -1)

# Calculamos la similitud del coseno
similitud = cosine_similarity(lista_riesgos_empresa_evaluada, lista_riesgos_todas_eval_sector) 

# Escalamos la similitud obtenida a un rango de 0% a 100%
porcentaje_similitud = round((similitud[0][0]) * 100, 1)

# Y por ultimo, mostramos el resultado final.
print("El porcentaje de similitud entre la evaluacion {} de la empresa {}\ny las demas evaluaciones del sector {} es del {}%".format(codigo_evaluacion, empresa_evaluada, sector_de_la_empresa, porcentaje_similitud))

umbral_satisfactorio = 50 # A partir de que porcentaje se considera correcta la evaluacion
if porcentaje_similitud >= umbral_satisfactorio:
    print("La evaluacion ha sido CORRECTA")
else:
    print("La evaluacion ha sido INCORRECTA")

El porcentaje de similitud entre la evaluacion 6 de la empresa 0019
y las demas evaluaciones del sector 11 es del 62.8%
La evaluacion ha sido CORRECTA
